In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
#setting
PROMPT_STYLE = 'AutoCOT1'
DATA_COUNT = 1
EXAMPLE_COUNT = 10

In [2]:
# Configs
RAW_PATH = '../data/1. raw/'
PLACED_PATH = '../data/2. placed/'
LABELED_PATH = '../data/3. labelled/'
PROMPT_PATH = '../data/4. prompt/'
PREPROCESSED_PATH = '../data/5. preprocessed/'
COMPARED_PATH = '../data/6. compared/'

BASE_PROMPT_FILE_PATH = f'./base_prompt/{PROMPT_STYLE}.txt'
PROMPT_FILE_PATH = PROMPT_PATH + f'{PROMPT_STYLE}.txt'

In [3]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

CompletedProcess(args=['node', 'generator.js', '200', '200'], returncode=0)

In [4]:
# Run placer.py
import placer

maps = placer.load_maps(path=RAW_PATH)
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, treasure_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps, path=PLACED_PATH)

In [5]:
# Run labeler.py
import labeler

labeler.label(placed_file_path=PLACED_PATH, labelled_file_path=LABELED_PATH, file_count=4, difficulty_curve_interval=5)

In [6]:
# Generate preprocessed
from preprocessed_data_generator import preprocessed_data_generator

preprocessed_data_generator(DATA_COUNT, EXAMPLE_COUNT, PREPROCESSED_PATH, BASE_PROMPT_FILE_PATH, LABELED_PATH, PROMPT_FILE_PATH)




find_final_map_keyword - Keyword positions: [137]
is_ascii_art_line - Line to check: '### Final ASCII Map' Result: False
is_ascii_art_line - Line to check: '' Result: False
is_ascii_art_line - Line to check: '```' Result: False
is_ascii_art_line - Line to check: '####################' Result: True
is_ascii_art_line - Line to check: '#....T.#...........#' Result: True
is_ascii_art_line - Line to check: '###/###############.#' Result: True
is_ascii_art_line - Line to check: '#..........R6....#.' Result: False
extract_ascii_art_map - Start index: 132, End index: 134
###/###############.#
#....T.#...........#
####################
Complete prompt saved to ../data/4. prompt/AutoCOT1.txt
find_final_map_keyword - Keyword positions: [115, 142]
is_ascii_art_line - Line to check: 'Final map looks like:' Result: False
is_ascii_art_line - Line to check: '' Result: False
is_ascii_art_line - Line to check: '- Boss (B) at (10, 12)' Result: False
is_ascii_art_line - Line to check: '- Player (P) at (15,

In [7]:
# Run comparator.py
import comparator

comparator.compare(preprocessed_path=PREPROCESSED_PATH, compared_path=COMPARED_PATH, file_count=1)

Object may be on inaccessible location.
Object may be on inaccessible location.
Object may be on inaccessible location.
Enemy may be on inaccessible location. (10, 6)
Enemy may be on inaccessible location. (10, 8)


In [8]:
# Get compared file
import os
import json

path = os.path.join(COMPARED_PATH)

filename_list = [
    filename
    for filename in os.listdir(path)
    if filename.endswith(".json") and os.path.isfile(os.path.join(path, filename))
]

path_list = [os.path.join(path, filename) for filename in filename_list]

compared_list = list()
for file_path in path_list:
    with open(file_path, "r") as file:
        compared_batch = json.load(file)
        for compared in compared_batch["map_list"]:
            compared_list.append(compared)

In [9]:
# Calculate difference
diff_dict = dict()

params = [
    "map_size",
    "room_count",
    "enemy_count",
    "treasure_count"
]

for param in params:
    diff_dict[param] = list()

for compared in compared_list:
    for param in params:
        try:
            diff_dict[param].append([abs(float(compared["after_params"][param][i]) - float(compared["before_params"][param][i])) for i in range(len(compared["after_params"][param]))])
        except:
            diff_dict[param].append(abs(float(compared["after_params"][param]) - float(compared["before_params"][param])))

In [10]:
# Calculate mean and standard deviation
import numpy as np

mean_dict = dict()
std_dict = dict()

for param in params:
    try:
        mean_dict[param] = np.mean(np.array(diff_dict[param]), axis=0)
        std_dict[param] = np.std(np.array(diff_dict[param]), axis=0)
    except:
        mean_dict[param] = np.mean(diff_dict[param])
        std_dict[param] = np.std(diff_dict[param])

In [11]:
# Print mean and standard deviation
print(mean_dict)
print(std_dict)

{'map_size': array([1., 2.]), 'room_count': 5.0, 'enemy_count': 0.0, 'treasure_count': 0.0}
{'map_size': array([0., 0.]), 'room_count': 0.0, 'enemy_count': 0.0, 'treasure_count': 0.0}


In [12]:
# Calculate playability

playability_list = list()
for i in range(len(compared_list)):
    playability_list.append(compared_list[i]["after_params"]["playability"])

playability_mean = np.mean(playability_list)

In [13]:
# Print playability
print(playability_mean)

0.0
